In [37]:
import numpy
import numpy as np

total_auc = {}
for split_part in np.arange(0.1, 1.0, 0.1):
    clf_ratio = np.round(split_part,2) 
    #save_file = 'Foursquare/split_data/'+str(clf_ratio)+'_train_file.txt'
    save_file = 'Gowalla/split_data/'+str(clf_ratio)+'_train_file.txt'
    text_id = {}
    id_text = {}
    
    i = 0
    def read_node_label(filename, i):
        fin = open(filename, 'r')
        while 1:
            l = fin.readline()
            if l == '':
                break
            vec = l.strip().split('\t')
            sid = vec[0]
            tid = vec[1]
            if sid not in text_id.keys():
                text_id[sid] = i
                i += 1
            if tid not in text_id.keys():
                text_id[tid] = i
                i += 1
        fin.close()

    read_node_label(save_file, i)
    #save_file = 'Foursquare/split_data/'+str(clf_ratio)+'_test_file.txt'
    save_file = 'Gowalla/split_data/'+str(clf_ratio)+'_test_file.txt'
    read_node_label(save_file, i)

    for i in range(len(text_id)):
        for j in text_id.keys():
            if text_id[j] == i:
                id_text[i] = j
                continue
                
                
                
    user_geo = {}
    loca_geo = {}
    X_train = []

    #label_file = 'Foursquare/split_data/'+str(clf_ratio)+'_train_file.txt'
    label_file = 'Gowalla/split_data/'+str(clf_ratio)+'_train_file.txt'
    
    def read_node_label(filename):
        fin = open(filename, 'r')
        while 1:
            l = fin.readline()
            if l == '':
                break
            vec = l.strip().split('\t')

            uid = vec[0] 
            lid = vec[1]
            geo = vec[2]

            if uid not in user_geo.keys():
                user_geo[uid] = geo

            if lid not in loca_geo.keys():
                loca_geo[uid] = geo

            tmp_geo = geo.split(',')

            X_train.append([float(tmp_geo[0]), float(tmp_geo[1])])
        fin.close()
    read_node_label(label_file)
        
    
    import numpy as np
    X_train = np.asarray(X_train)
    import matplotlib.pyplot as plt
    from matplotlib.colors import LogNorm
    from sklearn import mixture

    np.random.seed(0)
    clf = mixture.GaussianMixture(n_components=100, covariance_type='full')
    clf.fit(X_train)
    
    
    pre_X = {}

    #label_file = 'Foursquare/split_data/'+str(clf_ratio)+'_train_file.txt'
    label_file = 'Gowalla/split_data/'+str(clf_ratio)+'_train_file.txt'
    
    def read_node_label(filename):
        line = 0
        fin = open(filename, 'r')
        while 1:
            l = fin.readline()
            if l == '':
                break
            vec = l.strip().split('\t')

            uid = vec[0] 
            lid = vec[1]
            geo = vec[2].split(',')

            tmp_geo = []
            tmp_geo.append( [float(geo[0]), float(geo[1])] )
            tmp_geo = np.asarray(tmp_geo)

            log_prob = clf._estimate_log_prob(tmp_geo)
            i = 0
            tmp = log_prob[i,:]
            #tmp = tmp / sum(tmp)

            if uid in pre_X.keys():
                pre_X[uid] += tmp
            else:
                pre_X[uid] = tmp

            pre_X[lid] = tmp

        fin.close()

    read_node_label(label_file)
    
    
#     filename = 'Foursquare/split_data/'+str(clf_ratio)+'_GMM_embed.txt'

#     fout = open(filename, 'w')
#     for i in range(len(pre_X)):
#         fout.write("{}\n".format(' '.join([str(j) for j in pre_X[id_text[i]] / sum(pre_X[id_text[i]]) ]) ))

#     fout.close()    
    
    #filename = 'Foursquare/split_data/'+str(clf_ratio)+'_GMM_embed.txt'
    filename = 'Gowalla/split_data/'+str(clf_ratio)+'_GMM_embed.txt'
    fout = open(filename, 'w')
    for name, vec in pre_X.items(): 
        fout.write("{} {}\n".format(text_id[name], ' '.join([str(j) for j in pre_X[name] / sum(pre_X[name]) ]) ))

    fout.close()   
    
    #filename = 'Foursquare/split_data/'+str(clf_ratio)+'_GMM_tadw_embed.txt'
    filename = 'Gowalla/split_data/'+str(clf_ratio)+'_GMM_tadw_embed.txt'

    fout = open(filename, 'w')
    for name, vec in pre_X.items(): 
        fout.write("{} {}\n".format(text_id[name], ' '.join([str(j) for j in pre_X[name] / sum(pre_X[name]) ]) ))

    fout.close()    
    
    import random
    import numpy as np
    import os

    node2vec = {}
    random.seed(0)

#     f = open('Foursquare/split_data/'+str(clf_ratio)+'_GMM_embed.txt', 'rb')
#     for i, j in enumerate(f):
#         if j.decode() != '\n':
#             node2vec[i] = list(map(float, j.strip().decode().split(' ')))
            
    #f = open('Foursquare/split_data/'+str(clf_ratio)+'_GMM_embed.txt', 'rb')
    f = open('Gowalla/split_data/'+str(clf_ratio)+'_GMM_embed.txt', 'rb')
    for i, j in enumerate(f):
        if j.decode() != '\n':
            tmp = list(map(float, j.strip().decode().split(' ')))
            node2vec[tmp[0]] = tmp[1:]       

    #f1 = open(os.path.join('Foursquare/split_data/'+str(clf_ratio)+'_reorder_test.txt'), 'rb')
    f1 = open(os.path.join('Gowalla/split_data/'+str(clf_ratio)+'_reorder_test.txt'), 'rb')
    edges = [list(map(int, i.strip().decode().split('\t'))) for i in f1]
    nodes = list(set([i for j in edges for i in j]))
    a = 0
    b = 0
    for i, j in edges:
        if i in node2vec.keys() and j in node2vec.keys():
            dot1 = np.dot(node2vec[i], node2vec[j])
            random_node = random.sample(nodes, 1)[0]
            while random_node == j or random_node not in node2vec.keys():
                random_node = random.sample(nodes, 1)[0]
            dot2 = np.dot(node2vec[i], node2vec[random_node])
            if dot1 > dot2:
                a += 1
            elif dot1 == dot2:
                a += 0.5
            b += 1

    print("Auc value:", float(a) / b)
    
    total_auc[str(clf_ratio)] = float(a) / b

Auc value: 0.6813870050932831
Auc value: 0.6876290030517532
Auc value: 0.7807770650400327
Auc value: 0.7341998961644148
Auc value: 0.7512814798306218
Auc value: 0.7363304250927556
Auc value: 0.7855002237098694
Auc value: 0.7845411606792366
Auc value: 0.7610798764118928


In [38]:
# Gowalla
print(total_auc)

{'0.1': 0.6813870050932831, '0.2': 0.6876290030517532, '0.3': 0.7807770650400327, '0.4': 0.7341998961644148, '0.5': 0.7512814798306218, '0.6': 0.7363304250927556, '0.7': 0.7855002237098694, '0.8': 0.7845411606792366, '0.9': 0.7610798764118928}


In [36]:
# Forsquare
print(total_auc)

{'0.1': 0.6436106881589944, '0.2': 0.6645565952075834, '0.3': 0.6585232123406636, '0.4': 0.6882582750182459, '0.5': 0.6595915675809343, '0.6': 0.6753516047601875, '0.7': 0.6673879072003847, '0.8': 0.6911802586162485, '0.9': 0.6865694709185514}


In [ ]:
for i, name in pre_X.items():
    print(i, name)
    a = 1/0